# 🤖 사용자 질문 기반 숙소 리뷰 질의응답 시스템

In [ ]:

# !pip install sentence-transformers pandas scikit-learn


In [ ]:

import pandas as pd

# 감성 분석된 리뷰 데이터 (간단 샘플 300개)
df = pd.read_csv("숙소_리뷰_감성_분석_결과.csv")
df = df[['name', 'review_content', 'sentiment_tfidf']].dropna().reset_index(drop=True)
df = df[df['sentiment_tfidf'].isin(['긍정', '부정', '중립'])].head(300)


In [ ]:

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
review_embeddings = model.encode(df['review_content'], convert_to_tensor=True)


In [ ]:

from sklearn.metrics.pairwise import cosine_similarity

# 사용자 질문 입력
user_query = "조용하고 청결한 숙소 추천해줘"
query_vec = model.encode([user_query])

# 유사도 계산
cos_scores = cosine_similarity(query_vec, review_embeddings)[0]
top_idx = cos_scores.argsort()[::-1][:5]  # 상위 5개

print("🙋 사용자 질문:", user_query)
print("\n📍 가장 유사한 리뷰 5개:")
for i in top_idx:
    name = df.iloc[i]['name']
    review = df.iloc[i]['review_content']
    score = cos_scores[i]
    print(f"🏨 {name} ({score:.3f})\n→ {review}\n")
